In [2]:
import numpy as np
import math
import  re
import time
from tqdm import tqdm_notebook 
from collections import defaultdict
from sklearn.model_selection import train_test_split
from functools import reduce

Чтение полученного разбиения на кластеры и добавление в словарь

In [57]:
#f = open('fout_exemplars.txt','r')
f = open('C:\\Users\\Nuts\\Documents\\Visual Studio 2015\\Projects\\AffinityPropagation\\AffinityPropagation\\fout_exemplars.txt','r')
#соответсвие между юзерами и кластремами, ключ: user_id
id_cluster = defaultdict()

for line in tqdm_notebook(f):
    splited_line = re.split(r'\t',line)
    _id = int(splited_line[0])
    n_cluster = int(splited_line[1])
    id_cluster[_id] = n_cluster


A Jupyter Widget

In [58]:
len(set(id_cluster.values())) ## число полученых кластеров

47618

Чтение локаций и создание словаря, хранящего чекины каждого юзера

In [29]:
f = open("Gowalla_totalCheckins.txt", "r")

user_loc = defaultdict(list)

for line in tqdm_notebook(f):
    splited_line = re.split(r'\t',line)
    user_id = int(splited_line[0])
    location_id = int(splited_line[4])
    #добавление в словарь
    user_loc[user_id].append(location_id)

    
    

A Jupyter Widget

Разделение юзеров на тренировочных и тестовых(кому будут строиться рекомендации)

In [65]:

user_ids = user_loc.keys() #только те юзеры, которые имеют локации
train_ids,test_ids = train_test_split(list(user_ids), test_size = 0.010)


In [60]:
len(user_ids), len(test_ids) # юзеры, имеющие локации (все / тестовые)

(107092, 1071)

Создание словаря, хранящего чекины всех юзеров (из train), входящих в данный кластер.

key = номер кластера

In [52]:
cluster_loc = defaultdict(list)

for i in train_ids:
    cluster = id_cluster[i]
    locations = user_loc[i]
    for location in locations:
        cluster_loc[cluster].append(location)

In [53]:
len(cluster_loc) #число кластеров,попавших в train

34631

Подсчет качества рекомендаций

In [66]:

_sum = 0

for _id in tqdm_notebook(test_ids):
    start = time.time()
   
    cluster = id_cluster[_id]
    #полный список предсказанных локаций (несортированных по частоте встречаемости)
    predict_loc = cluster_loc[cluster]
    #print(_id, cluster, len(predict_loc))

    
    #сортировка по частоте встречаемости
    predict_loc = sorted(predict_loc,key=predict_loc.count,reverse=True)
    #print("after sort",_id, cluster, len(predict_loc))

    
    #оставить единственный экземпляр, сохраняя порядок сортировки
    predict_loc = reduce(lambda l,x:l+[x] if x not in l else l, predict_loc,[])
   
    
    #дополнить "ошибками" если меньше 10 локаций в кластере
    while len(predict_loc) < 10:
        predict_loc.append(-1)   
    
    #if len(predict_loc)>20000:
        #print("time: ", time.time() - start)
        
    # изменить значение в словаре, оставив только 10 популярных локаций     
    cluster_loc[cluster] = predict_loc[:10] 
     
    #истинные локации юзера
    current_loc = user_loc[_id]
    
    #прибавление числа пересечений 10 предсказанных с реальными локациями юзера
    _sum+= len(set(predict_loc[:10])&set(current_loc))
    #print("sum = ", _sum)
    

A Jupyter Widget

In [68]:
#максимальное число пересечений для всех тестовых юзеров
test_sum = 10 * len(test_ids) 

#отношение полученной суммы к максимально возможной
_sum / test_sum


0.25359477124183005

In [67]:
_sum

2716